In [1]:
import sys
import torch
from torch import nn

In [2]:
sys.path.append("..")

In [3]:
from torch.utils.data import DataLoader

In [4]:
from carca.data import CARCADataset, load_attrs, load_ctx, load_profiles
from carca.model import CARCA, BinaryCrossEntropy

In [5]:
attrs = load_attrs("video_games")
ctx = load_ctx("video_games")
user_ids, item_ids, profiles = load_profiles("video_games")

In [6]:
dataset = CARCADataset(user_ids, item_ids, profiles, attrs, ctx, 35, 100, "train")
loader = DataLoader(dataset, batch_size=64, shuffle=True)

In [7]:
p_x, o_x, p_q, o_q, y_true, mask = next(iter(loader))

In [8]:
model = CARCA(
    n_items=len(item_ids) + 1,
    d=50,
    g=250,
    n_ctx=next(iter(ctx.values())).shape[0],
    n_attrs=attrs.shape[1],
    H=1,
    p=0.5,
    B=1
)

In [ ]:
# TODO: SET TRAINING LOOP, DO NOT FORGET TO CORRECTLY SPECIFY HYPERPARAMS - L2 REGULARIZATION, OPTIMIZER, BATCH SIZE, LEARNING RATE, ...